In [182]:
import lyricsgenius
import nltk
import pandas as pd
import re

from decouple import config
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download(["stopwords", "punkt"])
genius = lyricsgenius.Genius(config("GENIUS_ACCESS_TOKEN"), remove_section_headers=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gppla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gppla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [209]:
artist = genius.search_artist("Damso", max_songs=10)
artist.save_lyrics(f"data/{artist.name}.json", sanitize=False)

Searching for songs by Damso...

Song 1: "Feu de bois"
Song 2: "Amnésie"
Song 3: "Θ. Macarena"
Song 4: "Γ. Mosaïque solitaire"
Song 5: "N. J Respect R"
Song 6: "Ε. Signaler"
Song 7: "Autotune"
Song 8: "Ζ. Kietu"
Song 9: "BruxellesVie"
Song 10: "Α. Nwaar Is The New Black"

Reached user-specified song limit (10).
Done. Found 10 songs.
Wrote data/Damso.json.


In [230]:
data = pd.read_json("data/JuL.json", typ="series")
data

alternate_names           [L'Ovni, Julien Mari, Jul Saint Jean de la Pue...
api_path                                                     /artists/74283
description               {'plain': 'Jul, stylisé JuL, de son nom comple...
facebook_name                                                   julOfficial
followers_count                                                         300
header_image_url          https://images.genius.com/f3f6f91406d8b491a8f4...
id                                                                    74283
image_url                 https://images.genius.com/6f7a11d70aa5ec81f5fd...
instagram_name                                                      juldetp
is_meme_verified                                                      False
is_verified                                                           False
name                                                                    JuL
translation_artist                                                    False
twitter_name

Clean up lyrics

In [231]:
songs = pd.DataFrame(data["songs"]).set_index("id")
def cleanup(lyrics):
    lyrics = " ".join(lyrics.split("\n")[2:]).lower()
    match = re.match("(.+?)(?=[0-9]+embed)", lyrics)
    if match:
        lyrics = match[1]
    return lyrics
songs["clean_lyrics"] = songs["lyrics"].map(cleanup)

all_lyrics = " ".join(songs["clean_lyrics"])
print(all_lyrics)

j'fume des grosses barres, j'm'étouffe il m'reste deux sticks, j'mets tout j'ai fait un rêve, 38 ans, j'disais au patron "j'guette où ?" j'sors de chez moi calé, j'entends : "akha, descente !" ça finit au palais, famille en pleurs, stressante trop de miss qui m'pistent autour de la piste, filles bien ou bitchs vite, y a embrouille, nique les videurs, même eux c'est des fils de p' embrouille, grosse droite, faut leur faire tomber le taser un poto s'est fait niquer, il veut revenir avec un fazer j'suis en chien malade, j'ai même pas pour faire 5-5 t'as assumé pour un braquage, ils t'en ont rajouté 25 embrouille, si si, j'suis là je sais même pas pourquoi maintenant tu veux nous niquer mais t'inquiètes, j'm'en fais pas pour moi  ce soir j'oublie tout j'cherche mon chemin, j'fais des détours ce soir j'oublie tout et quand j'repense à ce jour j'me dis qu'la vie est courte, on partira tous un jour alors j'm'en tape de vos discours, derrière le bonheur, moi je cours ce soir j'oublie tout j'ch

Tokenize text

In [232]:
sent_tokenizer = nltk.data.load("tokenizers/punkt/french.pickle")  # sentence tokenizer
w_tokenizer = RegexpTokenizer(r"\w'|\w+")  # word tokenizer

stops = set(stopwords.words("french")) | {"j'", "m'", "t'", "l'", "d'", "a", "c'", "s'"}
words = [word for sent in sent_tokenizer.tokenize(all_lyrics) for word in w_tokenizer.tokenize(sent) if not word in stops]
fd = nltk.FreqDist(words)

fd.tabulate(15)

    fais tchikita       si    comme    alors       ça     rien     wesh     tout    quand     vois     fait histoire     plus   quitte 
      99       57       46       45       43       40       35       34       33       29       28       25       25       25       24 
